In [16]:
from minio.error import S3Error
import fitz  # PyMuPDF
from datetime import datetime
from io import BytesIO
import pandas as pd
import re
import psycopg2
import import_ipynb
from Utils import conectar_minio,cargar_dfpaciente, generar_ruta_fecha,extraer_archivo_minio, insert_into_info_lab, agregar_fuente, agregar_observacion
import os
from dotenv import load_dotenv

In [17]:
# Cargar variables del archivo .env
load_dotenv()

True

In [18]:
minio_client = conectar_minio(
    endpoint=os.getenv("MINIO_ENDPOINT"),
    access_key=os.getenv("MINIO_ACCESS_KEY"),
    secret_key=os.getenv("MINIO_SECRET_KEY"),
    secure=False  # Usar HTTPS
)

Conexión exitosa a MinIO en localhost:9000
Buckets disponibles: ['yachay', 'yachay-bronze', 'yachay-landing']


In [19]:
ruta_fecha = generar_ruta_fecha() 

In [20]:
 #3. Extraer un archivo Excel/CSV/Parquet
df_datos = extraer_archivo_minio(
    minio_client=minio_client,
    bucket_name="yachay-bronze",
    ruta_archivo=f"laboratorios/{ruta_fecha}/laboratorio Monica Arcos/Monica Arcos.csv",
    tipo_archivo="csv"
)
print(df_datos)  # Mostrar las primeras filas del DataFrame extraído

        No   No Placa         Institución Administradora  \
0        1  2228327.0                   EMSSANAR EPS SAS   
1        2  2228328.0                   EMSSANAR EPS SAS   
2        3  2228412.0                   EMSSANAR EPS SAS   
3        4  2228413.0                   EMSSANAR EPS SAS   
4        5  2228414.0                   EMSSANAR EPS SAS   
...    ...        ...                                ...   
7127  7717  2327725.0  IPS CLINICA MARIANA TUQUERRES SAS   
7128  7718  2327726.0  IPS CLINICA MARIANA TUQUERRES SAS   
7129  7719  2327727.0  IPS CLINICA MARIANA TUQUERRES SAS   
7130  7720  2327728.0  IPS CLINICA MARIANA TUQUERRES SAS   
7131  7721  2327729.0  IPS CLINICA MARIANA TUQUERRES SAS   

                                   Institución Toma    Id Usuario  \
0                               RADIOLOGICO DEL SUR  CC1004631616   
1                                    HOSPITAL CIVIL    CC19371592   
2     IPS GASTROCENTER Y ESPECIALIDADES MEDICAS SAS    CC27388932   
3  

In [21]:
def transformar_df_lab_Monica(df_datos):
    """
    Transforma un DataFrame del laboratorio Mónica a la estructura de id_infolab
    
    Parámetros:
    - df_datos: DataFrame con los campos originales del laboratorio
    
    Retorna:
    - DataFrame transformado con la estructura deseada
    """
    
    # Crear copia para no modificar el original
    df_transformado = df_datos.copy()
    
    # Mapeo de campos
    mapeo_campos = {
        'Fecha Recibido': 'fecha_ingreso',
        'Fecha Resultado': 'fecha_informe',
        'Institución Administradora': 'eps',
        'Institución Toma': 'entidad',
        'Muestra de': 'comentario',
        'Dx Histopatológico': 'diagnostico',
        'Id Usuario': 'id_paciente',
        'Nombres Afiliado': 'Nombre del Paciente',
        'Edad': 'edad',
        'No': 'fuente'  # Usamos el número como fuente temporal
    }
    
    # Renombrar columnas
    df_transformado = df_transformado.rename(columns=mapeo_campos)
    
    # Campos que no tienen equivalente directo
    df_transformado['fecha_toma_muestra'] = None
    df_transformado['servicio'] = None
    df_transformado['descripcion_macroscopica'] = None
    df_transformado['descripcion_microscopica'] = None
    df_transformado['cie10'] = None
    df_transformado['sexo'] = None
    df_transformado['diagnostico_clinico'] = None
    
    # Formatear campo fuente
    #df_transformado['fuente'] = 'Laboratorio Mónica Arcos'
    
    return df_transformado

In [22]:
import pandas as pd

def eliminar_palabra_anios(df, nombre_columna):
    """
    Elimina la palabra 'años' de los valores en una columna específica
    
    Parámetros:
    - df: DataFrame de pandas
    - nombre_columna: Nombre de la columna a modificar (debe ser string)
    
    Retorna:
    - DataFrame con la columna modificada
    """
    # Verificar si la columna existe
    if nombre_columna not in df.columns:
        raise ValueError(f"La columna '{nombre_columna}' no existe en el DataFrame")
    
    # Hacer copia para no modificar el original
    df_modificado = df.copy()
    
    # Eliminar 'años' (case insensitive) y espacios extra
    df_modificado[nombre_columna] = df_modificado[nombre_columna].astype(str).str.replace(
        r'años|\s+', 
        '', 
        case=False, 
        regex=True
    )
    
    # Convertir a numérico si es posible
    try:
        df_modificado[nombre_columna] = pd.to_numeric(df_modificado[nombre_columna])
    except ValueError:
        pass  # Mantener como string si no se puede convertir
    
    return df_modificado

In [23]:
df_datos_transformado = transformar_df_lab_Monica(df_datos)


In [24]:
df_limpio = eliminar_palabra_anios(df_datos_transformado, 'edad')
df_limpio = agregar_fuente(df_limpio, '057')
df_limpio = agregar_observacion(df_limpio)

In [25]:
connection_params = {
    'dbname': os.getenv("DB_NAME"),
    'user': os.getenv("DB_USER"),
    'password': os.getenv("DB_PASSWORD"),
    'host': os.getenv("DB_HOST"),
    'port':os.getenv("DB_PORT")
}
insert_into_info_lab(df_limpio, connection_params)

Se insertaron 7132 filas en la tabla 'infolab'.


In [26]:
df_pacientes = df_limpio[['id_paciente', 'Nombre del Paciente', 'edad', 'sexo']].drop_duplicates()

In [27]:
def separar_tipo_numero_identificacion(df, columna):
    """
    Separa el tipo de identificación y el número de identificación a partir de una columna específica.

    Parámetros:
    - df: DataFrame de pandas.
    - columna: Nombre de la columna que contiene los datos de identificación.

    Retorna:
    - DataFrame con dos nuevas columnas: 'tipo_identificacion' y 'numero_identificacion'.
    """
    # Crear copia del DataFrame para no modificar el original
    df_resultado = df.copy()
    df_resultado[columna] = df_resultado[columna].astype(str).str.upper()  # Asegurarse de que la columna sea de tipo string
    # Extraer el tipo de identificación (CC, TI, PA) y el número de identificación
    df_resultado['tipo_de_identificacion'] = df_resultado[columna].str[:2].where(
        df_resultado[columna].str[:2].isin(['CC', 'TI', 'PA', 'AS','CE','PE','PT','RC','CN','MS']), None
    )
    df_resultado['identificacion'] = df_resultado[columna].str[2:]

    return df_resultado

In [28]:
df_pacientes = separar_tipo_numero_identificacion(df_pacientes, 'id_paciente')
print(df_pacientes.head())

    id_paciente                Nombre del Paciente edad  sexo  \
0  CC1004631616         REINA BASANTE ALEX ROLANDO   24  None   
1    CC19371592  ROSERO QUINGALAGUA HENRY MARCIANO   65  None   
2    CC27388932           LOPEZ BRAVO DEISY ESTELA   39  None   
3     CC5212496            LOPEZ AGUIRRE JULIO NEL   55  None   
4     CC1809960   VALENCIA QUITIAQUEZ JOSE CLIMACO   69  None   

  tipo_de_identificacion identificacion  
0                     CC     1004631616  
1                     CC       19371592  
2                     CC       27388932  
3                     CC        5212496  
4                     CC        1809960  


In [29]:
df_pacientes = df_pacientes.rename(columns={
    'Nombre del Paciente': 'nombre1',  
    'edad': 'edad',
    'sexo': 'sexo',
    'id_paciente': 'id_paciente'
})

In [30]:
cargar_dfpaciente(df_pacientes,'paciente', connection_params)

Insertados 0 registros nuevos en paciente.


(True, 0)